In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'bybit'
user = 'your_user_name'
settings = load_settings(exchange, user)
s = 'BTCUSD'
n_days = 7

In [ ]:
#load cache if any
#agg_trades = pd.read_csv('btcusdt_agg_trades_bybit_14_days_2021-01-07.csv').set_index('trade_id')
#adf = agg_trades

In [ ]:
#otherwise
#agg_trades = await load_trades(exchange, user, s, n_days)
#adf = agg_trades

In [ ]:
adf = agg_trades

In [ ]:
# look at jackrabbit results, if any
'''
rdf = pd.concat([pd.read_csv('jackrabbit_results_grid/2021-01-09T11:26:48_inverse.csv')])
column_names = ['grid_step', 'markups']
#column_names = ['default_qty', 'grid_step', 'markups', 'n_close_orders']
#column_names = ['default_qty', 'grid_step', 'margin_limit', 'markups', 'n_close_orders']

rdf.columns = column_names + list(rdf.columns[len(column_names):])
rdfs = rdf.sort_values('gain', ascending=[False])
rdfs = rdfs[~rdfs.gain.duplicated()]
rdfs.loc[:, 'pnl_over_max_margin_cost'] = rdfs.pnl_sum / rdfs.max_margin_cost
rdfs.head(30)
'''

In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ll = longs[longs.type == 'liq']
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        if 'close_price' in longs.columns:
            longs.close_price.plot(style='r--')
        if 'liq_price' in longs.columns:
            longs.liq_price.plot(style='k--')
        lc.price.plot(style='ro')
        ll.price.plot(style='gx')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        sl = shrts[shrts.type == 'liq']
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        if 'close_price' in shrts.columns:
            shrts.close_price.plot(style='b--')
        if 'liq_price' in shrts.columns:
            shrts.liq_price.plot(style='k--')
        sc.price.plot(style='bo')
        sl.price.plot(style='gx')

In [ ]:
# crop trades if desired
'''age_limit = (time() - 60 * 60 * 24 * 5) * 1000
adf = adf[adf.timestamp > age_limit]
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
n_days'''

In [ ]:
# prep df for backtesting
df = prep_df(adf)
print(len(df), len(df) / len(adf))
df.price.iloc[::100].plot()

In [ ]:
settings

In [ ]:
settings = {
    'default_qty': round(0.001 * 41000),
    'grid_step': 95,
    'leverage': 125,
    'maker_fee': 0.00018,
    'margin_limit': 50 / 41000,
    'markups': [0.00075, 0.0075],
    'min_qty': 1.0,
    'n_close_orders': 1,
    'n_entry_orders': 10,
    'price_step': 0.5,
    'qty_step': 1.0
}
print(json.dumps(settings, indent=4, sort_keys=True))

In [ ]:
start_time = time()
trades = backtest(df, settings)
print(f'seconds elapsed {time() - start_time:.2f}')

In [ ]:
# analyze results
tdf = pd.DataFrame(trades).set_index('trade_id')
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']

margin_max = (tdf.pos_size.abs() / tdf.pos_price).max() / settings['leverage']
biggest_pos_size = tdf.pos_size.abs().max()
pnl_sum = tdf.pnl.sum()
gain = (margin_max + pnl_sum) / margin_max
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
average_daily_gain = gain ** (1 / n_days)
closes = tdf[tdf.type == 'close']
print('margin_max', margin_max)
print('biggest qty', tdf.qty.abs().max())
print('pnl_sum', pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average_daily_gain', average_daily_gain)
print('n trades', len(tdf))
print('n closes', len(closes))
print('biggest_pos_size', biggest_pos_size)
print('approx pos_size limit', round(settings['margin_limit'] * settings['leverage'] * df.price.mean(), 2))

In [ ]:
plot_tdf_(df, tdf)

In [ ]:
# visualize behavior
step = 60
i = -step

In [ ]:
i += step
tdfc = tdf.iloc[i:i+step]
plot_tdf_(df, tdf.iloc[i:i+step])

In [ ]:
tdfc

In [ ]:
tdf.pnl.cumsum().plot()

In [ ]:
tdf.pos_size.plot()

In [ ]:
# inspect biggest trades
qty_abs = tdf.qty.abs().sort_values(ascending=False)
qty_abs.head(10)

In [ ]:
i = 4
iloc_ = tdf.index.get_loc(qty_abs.index[i])
iminus = 400
iplus = 20
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
ik = -1
stepp = 60

In [ ]:
ik += 1
tdfc.iloc[stepp * ik:stepp * (ik + 1)]